In [1]:
import sys
import datetime

import numpy as np
from scipy import integrate
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from struct import pack, unpack

sys.path.insert(0, "../")
from paths import FIG_DIR, DATA_DIR

In [2]:
def initialize_symmetric_tournament(m):
    if m % 2 != 1:
        raise ValueError("The value of m is even, please choose an odd value.")
    ssb = np.zeros((m, m))
    for k in range(1, int((m+1)/2)):
        for i in range(m):
            ssb[i, (i+k)%m] = 1
            ssb[i, (i-k)%m] = -1
    return ssb

In [ ]:
# intr_dict = {3: 0.24521831109433967, 5: 0.2486390358113208} # compare Notebook 5_1 and 5_3
# with open(DATA_DIR / '5_5_sim_symmetric.dat', 'wb') as file:
#     file.write(pack('d' * len(intr_dict) , *intr_dict.values()))

In [3]:
num_samples = 2.65 * 10**10 #1 sample corresponds to 3 lotteries, a triple!

with open(DATA_DIR / '5_5_sim_symmetric.dat', 'rb') as file:
    packed = file.read()
    read = list(unpack('d' * (len(packed) // 8), packed)) # 8 bytes per double
intr_dict = {2*i + 3: read[i] for i in range(len(read))}

for m in range(3 + 2*len(intr_dict), 16, 2):
    now = datetime.datetime.now()
    print(f"Starting {m} at {now.strftime('%Y-%m-%d %H:%M:%S')}")

    rng = np.random.default_rng(2025 + m)
    ssb_intr = initialize_symmetric_tournament(m)
    alpha = np.ones(m)
    batch_size = 10**7

    batch_results = []

    for i in range(len(batch_results)*batch_size, int(num_samples), batch_size):
        lotteries = rng.dirichlet(alpha, (3, batch_size))
        comp01 = (np.sum(lotteries[0,:,:] @ ssb_intr * lotteries[1,:,:], axis=1) > 0).astype(int)
        comp12 = (np.sum(lotteries[1,:,:] @ ssb_intr * lotteries[2,:,:], axis=1) > 0).astype(int)
        comp20 = (np.sum(lotteries[2,:,:] @ ssb_intr * lotteries[0,:,:], axis=1) > 0).astype(int)
        num_intr = np.sum(comp01 + comp12 + comp20 == 0) + np.sum(comp01 + comp12 + comp20 == 3)
        batch_results.append(num_intr/batch_size)

    now = datetime.datetime.now()
    print(f"Finished {m} at {now.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Results for m={m}: np.mean(batch_results)={np.mean(batch_results)}, np.std(batch_results)={np.std(batch_results)}, len(batch_results)={len(batch_results)}")
    intr_dict[m] = np.mean(batch_results)
    with open(DATA_DIR / '5_5_sim_symmetric.dat', 'wb') as file:
        file.write(pack('d' * len(intr_dict) , *intr_dict.values()))

Starting 13 at 2025-05-01 09:13:43
Finished 13 at 2025-05-01 13:29:23
Results for m=13: np.mean(batch_results)=0.24991388441509432, np.std(batch_results)=0.0001395371210316091, len(batch_results)=2650
Starting 15 at 2025-05-01 13:29:23
Finished 15 at 2025-05-01 18:14:58
Results for m=15: np.mean(batch_results)=0.24994523573584904, np.std(batch_results)=0.0001358840464983305, len(batch_results)=2650
